In [2]:
import pandas as pd
from sqlalchemy import create_engine
from config import dbuser, dbpasswd, dburi, dbport, dbname
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session


# CSV Productos - Data Frame

In [3]:
csv_productos = "Resources/Productos.csv"
productos_data_df = pd.read_csv(csv_productos, encoding="ISO-8859-1")
productos_data_df.head()

,ITE_ITEM_ID,ITE_ITEM_TITLE,ITE_STATUS,ITE_PICKING_TYPE,CAT_CATEG_ID_L1,CAT_CATEG_ID_L2,CAT_CATEG_ID_L3,TAG
0,601520411,Extracto Saw Palmeto O Sabal .,closed,drop_off,1403,178700,185387,No_SuperMarket
1,599332922,01 Paq Te Hamamelis 2 Hinojo Con Mapurite 2 Y ...,closed,drop_off,1403,178700,185387,No_SuperMarket
2,599662410,Extracto Cardo Mariano 30 Ml,active,drop_off,1403,178700,185387,No_SuperMarket
3,600001781,Extracto Castaña De Indias 30 Ml,active,drop_off,1403,178700,185387,No_SuperMarket
4,605466408,Pelotas Para El Agua Coloridas Para Bebés 200 ...,closed,default,1403,178700,185391,No_SuperMarket


In [4]:
# Clean productos_data
productos_data_df = productos_data_df.rename(columns = {'ITE_ITEM_ID': 'Item_ID', 'ITE_ITEM_TITLE': 'Description', 
                                                       'ITE_STATUS': 'Status', 'ITE_PICKING_TYPE': 'Deliver', 
                                                       'TAG': 'SuperMarket'})

productos_data_df = productos_data_df[['Item_ID', 'Description', 'Status','Deliver', 'CAT_CATEG_ID_L1','SuperMarket']].copy()

productos_data_df = productos_data_df[productos_data_df['Status']=='active']
productos_data_df.head()

,Item_ID,Description,Status,Deliver,CAT_CATEG_ID_L1,SuperMarket
2,599662410,Extracto Cardo Mariano 30 Ml,active,drop_off,1403,No_SuperMarket
3,600001781,Extracto Castaña De Indias 30 Ml,active,drop_off,1403,No_SuperMarket
6,600781357,Te Oregano,active,drop_off,1403,No_SuperMarket
7,600002255,Hierba Del Sapo Extracto,active,drop_off,1403,No_SuperMarket
8,604416984,Extracto De Tomillo,active,drop_off,1403,No_SuperMarket


# CSV Vendedores - Data Frame

In [5]:
csv_vendedores = "Resources/Vendedores.csv"
vendedores_data_df = pd.read_csv(csv_vendedores, encoding="ISO-8859-1")
vendedores_data_df.head()

,CUS_CUST_ID_SEL,CUS_NICKNAME,SEGMENTO
0,244515991,Teresa,LONG TAIL
1,270785336,Luis,LONG TAIL
2,16958172,Adrian,LONG TAIL
3,13572099,Malu,LONG TAIL
4,109512297,Rodrigo,LONG TAIL


In [6]:
# Change columns name to vendedores_data
vendedores_data_df = vendedores_data_df.rename(columns = {'CUS_CUST_ID_SEL': 'Seller_ID', 'CUS_NICKNAME': 'Seller'})
vendedores_data_df = vendedores_data_df[['Seller_ID', 'Seller']].copy()
vendedores_data_df.head()

,Seller_ID,Seller
0,244515991,Teresa
1,270785336,Luis
2,16958172,Adrian
3,13572099,Malu
4,109512297,Rodrigo


# CSV Ventas - Data Frame

In [7]:
csv_ventas = "Resources/Ventas.csv"
ventas_data_df = pd.read_csv(csv_ventas, encoding="ISO-8859-1")
ventas_data_df.head()

,DÍA,MES,WEEK,#_ID_Vendedor,#_ID_Producto,Valor_Promedio,Unidades
0,02/01/2018,Ene,W_01,244515991,601520411,199.00,1
1,02/01/2018,Ene,W_01,244515991,599332922,690.00,1
2,02/01/2018,Ene,W_01,244515991,599662410,199.00,1
3,02/01/2018,Ene,W_01,244515991,600001781,199.00,1
4,02/01/2018,Ene,W_01,270785336,605466408,210.93,1


In [8]:
# Clean ventas_data

ventas_data_df = ventas_data_df.rename(columns={'DÍA': 'Date', '#_ID_Vendedor': 'Seller_ID', '#_ID_Producto': 'Item_ID'})
ventas_data_df = ventas_data_df[['Date', 'Seller_ID', 'Item_ID', 'Valor_Promedio', 'Unidades']].copy()
ventas_data_df['Total_Sales'] = ventas_data_df['Valor_Promedio'] * ventas_data_df['Unidades']
ventas_data_df.head()

,Date,Seller_ID,Item_ID,Valor_Promedio,Unidades,Total_Sales
0,02/01/2018,244515991,601520411,199.00,1,199.00
1,02/01/2018,244515991,599332922,690.00,1,690.00
2,02/01/2018,244515991,599662410,199.00,1,199.00
3,02/01/2018,244515991,600001781,199.00,1,199.00
4,02/01/2018,270785336,605466408,210.93,1,210.93


# Upload tables into MYSQL

In [10]:
#session = Session(bind=engine)
Base = declarative_base()


In [11]:
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/sales_mexico_db")
Base.metadata.create_all(engine)

C:\Users\maluk\Anaconda3\envs\malucastellanos\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 484")
  result = self._query(query)


In [12]:
productos_data_df.to_sql(name='products', con=engine, if_exists='append', index=True)
vendedores_data_df.to_sql(name='vendedores', con=engine, if_exists='append', index=True)
ventas_data_df.to_sql(name='ventas', con=engine, if_exists='append', index=True)

In [13]:
engine.table_names()

['products', 'vendedores', 'ventas']

In [14]:
data=engine.execute('SELECT * FROM products limit 10')
for row in data:
    print(row)

(2, 599662410, 'Extracto Cardo Mariano 30 Ml', 'active', 'drop_off', 1403, 'No_SuperMarket')
(3, 600001781, 'Extracto Castaña De Indias 30 Ml', 'active', 'drop_off', 1403, 'No_SuperMarket')
(6, 600781357, 'Te Oregano', 'active', 'drop_off', 1403, 'No_SuperMarket')
(7, 600002255, 'Hierba Del Sapo Extracto', 'active', 'drop_off', 1403, 'No_SuperMarket')
(8, 604416984, 'Extracto De Tomillo', 'active', 'drop_off', 1403, 'No_SuperMarket')
(12, 600623850, '24g Boldo 65 Capsulas Envio Gratis', 'active', 'drop_off', 1403, 'No_SuperMarket')
(17, 608946723, 'Extracto Cardo Santo Envio Gratis', 'active', 'drop_off', 1403, 'No_SuperMarket')
(18, 608679180, 'Te Hierba Del Sapo', 'active', 'drop_off', 1403, 'No_SuperMarket')
(19, 608676919, 'Extracto Equinacea', 'active', 'drop_off', 1403, 'No_SuperMarket')
(20, 607748083, 'Sal De Mar  Organico Natural 2 Bolsas 1/2 Kilo  Envió Gratis', 'active', 'drop_off', 1403, 'No_SuperMarket')
